In [1]:
from GCForest import gcForest
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pickle
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from stackingFunc import *
import warnings
warnings.filterwarnings("ignore")
#用来计算程序运行时间
import datetime
starttime = datetime.datetime.now()


In [18]:
clfs = [gcForest(shape_1X=8064, window=512, stride=64,tolerance=0.0,n_cascadeRF=1, 
               min_samples_mgs=0.1, min_samples_cascade=0.1,n_jobs=19),
       gcForest(shape_1X=8064, window=512, stride=64,tolerance=0.0,n_cascadeRF=1, 
               min_samples_mgs=0.1, min_samples_cascade=0.1,n_jobs=19),]

#读取Y
y = pickle.load(open("./dump_file/all_df_y_2c","rb"))
print("y.shape:",y.shape)

#读取X
for eeg_CH in range(1,2,1):
    file_path = "./dump_file/CH{}_df_EEG_x".format(eeg_CH)
    locals()['CH{}_df_EEG_x'.format(eeg_CH)] = pickle.load(open(file_path,"rb"))
    X = locals()['CH{}_df_EEG_x'.format(eeg_CH)]
    for i in X.values:
        for j in i:
            if j==np.NaN:
                print(j)
    X.fillna(X.median(axis=1),inplace=True)   
    X = X.astype(np.float32)
    min_max_scaler = MinMaxScaler(feature_range=(-1,1))
    X = min_max_scaler.fit_transform(X)
    y = y.values
    
'''切分一部分数据作为测试集'''
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.3, random_state=2018)

y.shape: (1280, 1)


In [15]:
for i in X:
    for j in i:
        if j==np.NaN:
            print(j)


In [13]:
trainSet_2nd,testSet_2nd = get_stacking(clfs[0],x_train=X_tr,y_train=y_tr,x_test=X_te,n_folds=5)

Slicing Sequence...
Training MGS Random Forests...
Adding/Training Layer, n_layer=1
Layer validation accuracy = 0.6527777777777778
Adding/Training Layer, n_layer=2
Layer validation accuracy = 0.6527777777777778
Slicing Sequence...
Slicing Sequence...
Slicing Sequence...
Training MGS Random Forests...
Adding/Training Layer, n_layer=1


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').